# Process

- 단어 추출(토크나이저 재구성)
- 사전 만들기
- 문서 - 단어 - 벡터화(매트릭스 만들기)
- 시기 별(1개월 단위로) "박근혜", "대통령", "당선인" 등 박근혜 전 대통령을 지칭하는 키워드와 유사어 추출

- 워드클라우드로 구현

---

### SETUP

In [125]:
import pandas as pd
import re
import matplotlib.pyplot as plt

%matplotlib inline

In [30]:
!ls ./lexicon

expressive-type.csv       polarity.csv              subjectivity-type.csv
intensity.csv             readme.txt
nested-order.csv          subjectivity-polarity.csv


In [15]:
data_raw = pd.read_pickle('cs_whole.pkl')
data_raw.tail()

,date,title,body
1017,2017.03.06,"[사설] 운명의 일주일, '탄핵' '기각' 이후가 더 중요하다",이르면 이번 주 후반 박근혜 대통령 탄핵 심판 사건에 대한 헌재 결정이 나올 가능성...
1018,2017.03.07,[사설] 중국 땅인지 착각게 한 롯데 앞 촛불 시위대,지난 4일 밤 광주 롯데백화점 앞 촛불 시위 사진은 눈을 의심케 했다. 수백 명이 ...
1019,2017.03.07,"[사설] 특검, '최순실 국정농단'의 本流 꿰뚫은 수사였나",박영수 특검팀이 6일 수사 결과를 발표했다. 그러나 평가는 엇갈린다. 구속기소된 이...
1020,2017.03.09,"[사설] 10일 탄핵 심판 선고, 모두 自重하고 또 自制하자",박근혜 대통령 탄핵 심판 선고 기일이 10일로 확정됐다. 헌재 재판관 8명이 8일 ...
1021,2017.03.10,"[사설] 오늘 시험대 오르는 대한민국, '역사적 승복'으로 위기 끝내자",헌법재판소가 오늘 11시 박근혜 대통령 탄핵심판 사건을 선고한다. 작년 10월 5일...


### preprocessing

In [22]:
# erase \n 

In [23]:
data_raw.body[0][:1000]

"19일 실시된 대통령선거에서 새누리당 박근혜 후보가 투표자 3072만명 중 51% 남짓 1600만표가량을 얻어 18대 대통령에 당선됐다. 민주통합당 문재인 후보는 48% 남짓 1500만표가량을 얻었다. 1987년 직선제 개헌 이후 득표율 50%를 넘긴 대통령은 이번이 처음이다. 70%를 약간 넘을 것으로 예상했던 투표율은 75.8%까지 올랐다. 대한민국 건국 이후 첫 여성 대통령이 탄생했다. 박 후보의 당선으로 투표율이 높으면 야당이 유리하다는 속설(俗說)도 깨졌다.\n경쟁자 지지 국민의 박탈감 헤아려야\n\n정권 재창출보다 정권 교체를 바라는 여론이 더 높은 가운데 치러진 이번 대선은 박 후보에게 버거운 선거였다. 선거 초반 한동안 유지되던 박근혜 대세론은 무소속 안철수 후보 등장으로 몇 차례 크게 흔들렸다. 박 후보는 박근혜·문재인·안철수 3자 가상(假想) 대결에선 늘 1위를 지켰으나 야당 단일 후보에겐 밀리는 결과가 몇 번이나 나타났다. 안 후보 사퇴 이후 박 후보에게 10%포인트 이상 뒤지던 문 후보의 지지율이 꾸준히 올라가 선거 이틀 전에는 뒤집히기도 했다.\n\n이 순간 박 당선인에게 가장 절실한 자세는 자신을 지지한 1600만 국민과 함께 자신의 경쟁자에게 표를 던진 1500만 국민의 마음을 정확히 읽고 그들을 진정으로 끌어안는 것이다. 이명박 정부 5년 동안 우리나라는 미국발(發) 금융위기·유럽발 재정위기 속에서도 세계 각국 가운데 거시(巨視)경제지표가 가장 빨리 호전되고 한 사회의 소득 불평등 정도를 가리키는 지니계수도 노무현 정부 동안 계속 악화하다 2009년 0.320에서 2011년 0.313으로 개선됐다. 그럼에도 국민은 이런 지표 개선과 관계없이 어쩌면 오히려 그것 때문에 더 큰 경제적 고통과 상대적 박탈감을 느껴왔다. 대기업의 경기가 좋아지면, 고소득자의 소득이 높아지면, 그 효과가 형편이 어려운 사람들에게도 퍼져간다는 이른바 '낙수(落水)효과'는 현실과 거리가 먼 것으로 드러났다.\n\nDB손해보험 다이렉트 바로가기\n젊은이들은 대

In [26]:
data_raw.body = data_raw.body.apply(lambda x : re.sub("\n", " ", x) if re.findall("\n", x) else x)
data_raw.body[0][:1000]

"19일 실시된 대통령선거에서 새누리당 박근혜 후보가 투표자 3072만명 중 51% 남짓 1600만표가량을 얻어 18대 대통령에 당선됐다. 민주통합당 문재인 후보는 48% 남짓 1500만표가량을 얻었다. 1987년 직선제 개헌 이후 득표율 50%를 넘긴 대통령은 이번이 처음이다. 70%를 약간 넘을 것으로 예상했던 투표율은 75.8%까지 올랐다. 대한민국 건국 이후 첫 여성 대통령이 탄생했다. 박 후보의 당선으로 투표율이 높으면 야당이 유리하다는 속설(俗說)도 깨졌다. 경쟁자 지지 국민의 박탈감 헤아려야  정권 재창출보다 정권 교체를 바라는 여론이 더 높은 가운데 치러진 이번 대선은 박 후보에게 버거운 선거였다. 선거 초반 한동안 유지되던 박근혜 대세론은 무소속 안철수 후보 등장으로 몇 차례 크게 흔들렸다. 박 후보는 박근혜·문재인·안철수 3자 가상(假想) 대결에선 늘 1위를 지켰으나 야당 단일 후보에겐 밀리는 결과가 몇 번이나 나타났다. 안 후보 사퇴 이후 박 후보에게 10%포인트 이상 뒤지던 문 후보의 지지율이 꾸준히 올라가 선거 이틀 전에는 뒤집히기도 했다.  이 순간 박 당선인에게 가장 절실한 자세는 자신을 지지한 1600만 국민과 함께 자신의 경쟁자에게 표를 던진 1500만 국민의 마음을 정확히 읽고 그들을 진정으로 끌어안는 것이다. 이명박 정부 5년 동안 우리나라는 미국발(發) 금융위기·유럽발 재정위기 속에서도 세계 각국 가운데 거시(巨視)경제지표가 가장 빨리 호전되고 한 사회의 소득 불평등 정도를 가리키는 지니계수도 노무현 정부 동안 계속 악화하다 2009년 0.320에서 2011년 0.313으로 개선됐다. 그럼에도 국민은 이런 지표 개선과 관계없이 어쩌면 오히려 그것 때문에 더 큰 경제적 고통과 상대적 박탈감을 느껴왔다. 대기업의 경기가 좋아지면, 고소득자의 소득이 높아지면, 그 효과가 형편이 어려운 사람들에게도 퍼져간다는 이른바 '낙수(落水)효과'는 현실과 거리가 먼 것으로 드러났다.  DB손해보험 다이렉트 바로가기 젊은이들은 대학을 졸업해도 

In [35]:
corpus = data_raw.body.values
corpus = list(corpus)
corpus[0][:1000]

"19일 실시된 대통령선거에서 새누리당 박근혜 후보가 투표자 3072만명 중 51% 남짓 1600만표가량을 얻어 18대 대통령에 당선됐다. 민주통합당 문재인 후보는 48% 남짓 1500만표가량을 얻었다. 1987년 직선제 개헌 이후 득표율 50%를 넘긴 대통령은 이번이 처음이다. 70%를 약간 넘을 것으로 예상했던 투표율은 75.8%까지 올랐다. 대한민국 건국 이후 첫 여성 대통령이 탄생했다. 박 후보의 당선으로 투표율이 높으면 야당이 유리하다는 속설(俗說)도 깨졌다. 경쟁자 지지 국민의 박탈감 헤아려야  정권 재창출보다 정권 교체를 바라는 여론이 더 높은 가운데 치러진 이번 대선은 박 후보에게 버거운 선거였다. 선거 초반 한동안 유지되던 박근혜 대세론은 무소속 안철수 후보 등장으로 몇 차례 크게 흔들렸다. 박 후보는 박근혜·문재인·안철수 3자 가상(假想) 대결에선 늘 1위를 지켰으나 야당 단일 후보에겐 밀리는 결과가 몇 번이나 나타났다. 안 후보 사퇴 이후 박 후보에게 10%포인트 이상 뒤지던 문 후보의 지지율이 꾸준히 올라가 선거 이틀 전에는 뒤집히기도 했다.  이 순간 박 당선인에게 가장 절실한 자세는 자신을 지지한 1600만 국민과 함께 자신의 경쟁자에게 표를 던진 1500만 국민의 마음을 정확히 읽고 그들을 진정으로 끌어안는 것이다. 이명박 정부 5년 동안 우리나라는 미국발(發) 금융위기·유럽발 재정위기 속에서도 세계 각국 가운데 거시(巨視)경제지표가 가장 빨리 호전되고 한 사회의 소득 불평등 정도를 가리키는 지니계수도 노무현 정부 동안 계속 악화하다 2009년 0.320에서 2011년 0.313으로 개선됐다. 그럼에도 국민은 이런 지표 개선과 관계없이 어쩌면 오히려 그것 때문에 더 큰 경제적 고통과 상대적 박탈감을 느껴왔다. 대기업의 경기가 좋아지면, 고소득자의 소득이 높아지면, 그 효과가 형편이 어려운 사람들에게도 퍼져간다는 이른바 '낙수(落水)효과'는 현실과 거리가 먼 것으로 드러났다.  DB손해보험 다이렉트 바로가기 젊은이들은 대학을 졸업해도 

In [31]:
# 단어 추출

In [36]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    max_left_length=10,
    max_right_length=6,
    min_frequency=5
)

word_extractor.train(corpus)

training was done. used memory 0.238 Gb


In [38]:
scores = word_extractor.word_scores()

all cohesion probabilities was computed. # words = 27258
all branching entropies was computed # words = 42263
all accessor variety was computed # words = 42263


In [39]:
scores['박근혜']

Scores(cohesion_forward=0.9183683615125757, cohesion_backward=0.24505110018405926, left_branching_entropy=2.3429356954455187, right_branching_entropy=0.90776970980902, left_accessor_variety=53, right_accessor_variety=13, leftside_frequency=1131, rightside_frequency=72)

In [40]:
# 토크나이징

In [195]:
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer
from sklearn.feature_extraction.text import CountVectorizer

def make_dataset(corpus):

    # scores dict로 cohesion scores, ltokenizer 만들기
    cohesion_scores = {word:score.cohesion_forward for word, score in scores.items()}
    ltokenizer = LTokenizer(scores=cohesion_scores)

    def l_tokenizer(word):
        return ltokenizer.tokenize(word, remove_r=True)

    # ltokenizer를 활용해서 벡터라이저 만들기
    vectorizer = CountVectorizer(
        tokenizer=l_tokenizer,
    )

    # x 만들기
    x = vectorizer.fit_transform(corpus)
    
    # vocab2int, int2vocab 만들기
    vocab2int = vectorizer.vocabulary_
    int2vocab = [
        word for word, index in sorted(vocab2int.items(), key=lambda x:x[1])
    ]
    
    return x, vocab2int, int2vocab

x, vocab2int, int2vocab = make_dataset(corpus)

def word2int(word):
    return vocab2int.get(word, -1)

def int2word(idx):
    if 0 <= idx < len(int2vocab):
        return int2vocab[idx]
    return None

### keyword extraction

In [101]:
x = x_sparse.copy()

In [102]:
keyword_list = ["박근혜", "대통령", "당선인"]

for word in keyword_list:

    word_idx = word2int(word)
    positive_document = x[:, word_idx].nonzero()[0]

    print('{:3,} 중 {:3,} 문서가 "{}" 라는 단어를 가지고 있습니다.'.format(x.shape[0], len(positive_document), word))

1,022 중 965 문서가 "박근혜" 라는 단어를 가지고 있습니다.
1,022 중 907 문서가 "대통령" 라는 단어를 가지고 있습니다.
1,022 중  93 문서가 "당선인" 라는 단어를 가지고 있습니다.


In [103]:
word = "박근혜"
word_idx = word2int(word)
positive_document = x[:, word_idx].nonzero()[0]

print('{:3,} 중 {:3,} 문서가 "{}" 라는 단어를 가지고 있습니다.'.format(x.shape[0], len(positive_document), word))

from collections import Counter

y = [1 if i in positive_document else -1 for i in range(x.shape[0])]
print(Counter(y))

1,022 중 965 문서가 "박근혜" 라는 단어를 가지고 있습니다.
Counter({1: 965, -1: 57})


In [99]:
# train dataset 만들기

In [104]:
def get_train_data(word):
    
    word_idx = word2int(word) # word2int 함수 필요
    positive_document = x[:, word_idx].nonzero()[0] # x 데이터셋 필요
    
    def get_label(i):
        return 1 if i in positive_document else -1
    
    y_train = [get_label(i) for i in range(x.shape[0])]
    
    (row, col) = x.nonzero()
    data = x.data
    
    row_ = []
    col_ = []
    data_ = []
    
    for r, c, d in zip(row, col, data):
        if c == word_idx:
            continue
        row_.append(r)
        col_.append(c)
        data_.append(d)
        
    from scipy.sparse import csr_matrix
    x_train = csr_matrix((data_, (row_, col_)))
    
    return x_train, y_train

x_train, y_train = get_train_data('박근혜')

In [106]:
# 유사어 추출

In [273]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1', C=10)
logistic_l1.fit(x_train, y_train)

def lasso_keyword(word, C=20, topk=20):
    if not (word in vocab2int):
        return []
    
    x_train, y_train = get_train_data(word)
    logistic_l1 = LogisticRegression(penalty='l1', C=C)
    logistic_l1.fit(x_train, y_train)
    
    idx_coef = enumerate(logistic_l1.coef_.reshape(-1))
    sorted_coefficients = sorted(idx_coef, key=lambda x:-x[1])
    
    # filtering keyword
    keywords = [word_idx for word_idx, coef in sorted_coefficients[:topk] 
                if coef > 0.001]
    
    # decode idx to str
    keywords = [int2word(word_idx) for word_idx in keywords]
    return keywords

print(lasso_keyword('박근혜'))

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

In [111]:
# idx_coef = enumerate(logistic_l1.coef_.reshape(-1))
# sorted_coefficients = sorted(idx_coef, key=lambda x:-x[1])

# for word_idx, coef in sorted_coefficients[:10]:
#     if coef < 0.001: break
#     print(int2word(word_idx))

있어
당선인
계엄
한·미
촛불
요구
도발
관계
허위
한다.


### 노트

1. customized_tokenizer가 명사, 용언만 가져오지 않기 때문에 "있어", "한다."처럼 불필요한 단어가 상위에 등장한다.
> 사설 분석에 적합한 토크나이저가 필요하다.

2. 시기 별 키워드 분석을 위해서는 월 단위로 기사 분류.
3. 월(혹은 분기) 단위로 워드클라우드 구현.
4. 전체 사설 데이터를 활용하는게 더 나을 수 있다.
5. 옛날 신문 같은 경우에는 한자를 한글로 변환해야 한다.

In [122]:
data_revise = data_raw.copy()
data_revise.date = data_revise.date.apply(lambda x : x[:7])
data_revise.head()

,date,title,body
0,2012.12,"[사설] 박근혜 당선인, 겸허하게 온 국민 껴안는 걸로 시작하라",19일 실시된 대통령선거에서 새누리당 박근혜 후보가 투표자 3072만명 중 51% ...
1,2012.12,"[사설] 선거 여론조사, '公表 금지 기간' 없애거나 더 단축을",국민은 여론조사 공표가 금지된 지난 13일부터 18일까지 6일 동안 여론 흐름을 전...
2,2012.12,"[사설] 한국 진보 左派, 進化하지 않으면 몰락한다",민주당 문재인 후보는 범(汎)야권 후보를 완벽하게 단일화하고 투표율이 자신들이 야당...
3,2012.12,"[사설] 政權 인수, 소리 내지 말고 실무적으로 하라",곧 박근혜대통령직인수위원회가 발족한다. 역대 대통령직인수위에 대한 국민 기억은 그다...
4,2012.12,[사설] '5060世代 지혜' 못지않게 '2030 氣 살리기'도 중요하다,19일의 대선에서 50대(代) 투표율이 89.9%였던 것으로 방송사 출구 조사에서 ...


In [178]:
# 기간 별 텍스트 데이터 취합

In [180]:
month = np.unique(data_revise.date)
month

array(['2012.12', '2013.01', '2013.02', '2013.03', '2013.04', '2013.05',
       '2013.06', '2013.07', '2013.08', '2013.09', '2013.10', '2013.11',
       '2013.12', '2014.01', '2014.02', '2014.03', '2014.04', '2014.05',
       '2014.06', '2014.07', '2014.08', '2014.09', '2014.10', '2014.11',
       '2014.12', '2015.01', '2015.02', '2015.03', '2015.04', '2015.05',
       '2015.06', '2015.07', '2015.08', '2015.09', '2015.10', '2015.11',
       '2015.12', '2016.01', '2016.02', '2016.03', '2016.04', '2016.05',
       '2016.06', '2016.07', '2016.08', '2016.09', '2016.10', '2016.11',
       '2016.12', '2017.01', '2017.02', '2017.03', ': 2016.'],
      dtype=object)

In [181]:
data_merged = pd.DataFrame(columns=['date', 'title', 'body'])
data_merged

,date,title,body


In [183]:
for dates in month:

    # data revise에서 샘플링
    sample = data_revise[data_revise.date == dates]

    titles = sample.title.values
    titles = list(titles)
    titles = ''.join(titles)
    
    articles = sample.body.values
    articles = list(articles)
    articles = ''.join(articles)

    data = {
        "date" : dates,
        "title" : titles,
        "body" : articles
    }
    
    data_merged.loc[len(data_merged)] = data

data_merged

,date,title,body
0,2012.12,"[사설] 박근혜 당선인, 겸허하게 온 국민 껴안는 걸로 시작하라[사설] 선거 여론조...",19일 실시된 대통령선거에서 새누리당 박근혜 후보가 투표자 3072만명 중 51% ...
1,2013.01,"[사설] ""남북 대결 상태 해소"" 북한 하기에 달려[사설] 극빈층 3% 진료비, 상...","김정은 북한 노동당 제1비서는 1일 북한 매체를 통해 중계된 육성 신년사에서 ""나라..."
2,2013.02,[사설] 청문회 통과할 후보 먼저 찾고 제도 개선은 추후로[사설] 朴 당선인에게 속...,"박근혜 대통령 당선인은 1월 31일 인사청문회에 대해 ""공직자의 신상에 대한 문제는..."
3,2013.03,[사설] 與黨 지도부가 움직여야 할 때다[사설] 청문회장의 '5·16 愚問愚答' 보...,새누리당 황우여 대표와 이한구 원내대표는 2월 28일 최고위원 회의 모두 발언에서 ...
4,2013.04,[사설] 세금 야박하게 긁어낸다고 福祉 비용 댈 수 있을까[사설] 핵 끌어안고 경제...,정부가 올해 예산 편성을 하면서 교통범칙금 징수 목표 액수를 작년보다 12%(900...
5,2013.05,[사설] 단발성 규제 완화론 '投資 붐' 일으킬 수 없다[사설] 긴박한 한반도 정세...,정부는 1일 열린 무역투자진흥회의에서 그동안 허가를 미루고 있던 에쓰오일·SK종합화...
6,2013.06,[사설] 대통령 지지율을 국정 動力으로 연결하려면[사설] 고위 공무원 여성 비율 4...,4일 박근혜 대통령은 취임 100일을 맞는다. 취임 100일을 앞두고 미디어리서치가...
7,2013.07,[사설] 韓·中 관계는 새로운 시대의 門 앞에 서 있다[사설] 동북아 전환기 속 진...,박근혜 대통령이 4일간의 중국 방문을 마치고 30일 귀국했다. 박 대통령은 새 대통...
8,2013.08,"[사설] 새누리 지휘부, 정국 主導 정치력 발휘하라[사설] '송전탑 반대' 전국 확...","여야는 4일 국정원 국정조사 정상화 협상을 재개, 국정원 보고를 예정대로 듣기로 하..."
9,2013.09,"[사설] 與野, 정기국회 허송하고 막판 벼락치기 할 건가[사설] 기초연금 축소, 먼...",정기국회가 2일 문을 열었지만 여야는 통합진보당 이석기 의원 체포동의안 처리에 합의...


In [194]:
data_merged.to_pickle('cs_data_merged_v1.pkl')

### 데이터셋 만들기

In [242]:
corpus = data_merged.body.values
corpus = list(corpus)

print(len(corpus), "\n", type(corpus))

53 
 <class 'list'>


In [274]:
from soynlp.tokenizer import LTokenizer, MaxScoreTokenizer
from sklearn.feature_extraction.text import CountVectorizer

def make_dataset(corpus):

    # scores dict로 cohesion scores, ltokenizer 만들기
    cohesion_scores = {word:score.cohesion_forward for word, score in scores.items()}
    ltokenizer = LTokenizer(scores=cohesion_scores)

    def l_tokenizer(word):
        return ltokenizer.tokenize(word, remove_r=True)

    # ltokenizer를 활용해서 벡터라이저 만들기
    vectorizer = CountVectorizer(
        tokenizer=l_tokenizer,
    )

    # x 만들기
    x = vectorizer.fit_transform(corpus)
    
    # vocab2int, int2vocab 만들기
    vocab2int = vectorizer.vocabulary_
    int2vocab = [
        word for word, index in sorted(vocab2int.items(), key=lambda x:x[1])
    ]
    
    return x, vocab2int, int2vocab

x, vocab2int, int2vocab = make_dataset(corpus)

def word2int(word):
    return vocab2int.get(word, -1)

def int2word(idx):
    if 0 <= idx < len(int2vocab):
        return int2vocab[idx]
    return None

### x train, y train 으로 구성

In [275]:
def get_train_data(word):
    
    word_idx = word2int(word) # word2int 함수 필요
    positive_document = x[:, word_idx].nonzero()[0] # x 데이터셋 필요
    
    def get_label(i):
        return 1 if i in positive_document else -1
    
    y_train = [get_label(i) for i in range(x.shape[0])]
    
    (row, col) = x.nonzero()
    data = x.data
    
    row_ = []
    col_ = []
    data_ = []
    
    for r, c, d in zip(row, col, data):
        if c == word_idx:
            continue
        row_.append(r)
        col_.append(c)
        data_.append(d)
        
    from scipy.sparse import csr_matrix
    x_train = csr_matrix((data_, (row_, col_)))
    
    return x_train, y_train

x_train, y_train = get_train_data('박근혜')

### 유사어 추출

In [276]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1', C=10)
logistic_l1.fit(x_train, y_train)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

In [277]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1', C=10)
logistic_l1.fit(x_train, y_train)

def lasso_keyword(word, C=20, topk=20):
    if not (word in vocab2int):
        return []
    
    x_train, y_train = get_train_data(word)
    logistic_l1 = LogisticRegression(penalty='l1', C=C)
    logistic_l1.fit(x_train, y_train)
    
    idx_coef = enumerate(logistic_l1.coef_.reshape(-1))
    sorted_coefficients = sorted(idx_coef, key=lambda x:-x[1])
    
    # filtering keyword
    keywords = [word_idx for word_idx, coef in sorted_coefficients[:topk] 
                if coef > 0.001]
    
    # decode idx to str
    keywords = [int2word(word_idx) for word_idx in keywords]
    return keywords

print(lasso_keyword('박근혜'))

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

### 노트

- 유사어는 특정 키워드를 동일하게 가지고 있는 문서 안에 있는 키워드를 유사어로 지정한다. 
> 그러므로 차라리 월 별 코퍼스의 키워드를 분석하는 게 더 나을 것 같다.

- 대부분 "박근혜" 키워드를 가지고 있으므로, 연관어 / 키워드를 분류하는게 애초에 불가능하다. Lasso를 사용하는 경우는, logistic regression은 대표벡터를 학습해야 하는데 위 코퍼스는 전부 다 "박근혜"라는 단어를 포함하고 있어, 2개 이상 집단으로 나뉘지 않아 2개 이상의 대표벡터가 존재하지 않기 때문이다.
> 그렇다면 "박근혜"를 코퍼스에서 제거한 후 연관어, 키워드를 분석하면 어떨까?

### 키워드 추출(선회)

In [234]:
x

<1x3101 sparse matrix of type '<class 'numpy.int64'>'
	with 3101 stored elements in Compressed Sparse Row format>

In [271]:
from soykeyword.proportion import MatrixbasedKeywordExtractor

proportion_based_extractor = MatrixbasedKeywordExtractor(
    min_tf=0, 
    min_df=0,
    verbose=True)

proportion_based_extractor.train(x, int2vocab)

MatrixbasedKeywordExtractor trained


In [272]:
keywords = proportion_based_extractor.extract_from_word('박근혜')
keywords[:10]

ZeroDivisionError: division by zero

### 노트

- "박근혜"가 포함된 문서의 키워드를 추출하려니, 확률 기반 추출방법도 제대로 되지 않는다. 
> 우선 가장 간단한 방법인 워드 클라우드부터 시도해봐야겠다. 월 별 가장 높은 빈도가 나오는 명사, 용언만 추출해서 보여주도록.